# Training a segmentation model with data augmentation

In this notebook, we will use Nobrainer to augment training data which will be useful to learn more robust model for brain extraction task. Nobrainer provides several methods of augmenting volumetric data including spatial and intensity transforms.Augmentation is useful when the training data is insufficient to train large convolutional neural networks. Augmentation also improves robustness of the network and improves generalization to different datasets. 


In the following cells, we will:

1. Get sample T1-weighted MR scans as features and FreeSurfer segmentations as labels.
    - We will binarize the FreeSurfer to get a precise brainmask.
2. Convert the data to TFRecords format.
3. Create two Datasets of the features and labels.
    - One dataset will be for training and the other will be for evaluation.
    - Set a sequence of augmentations for training 
4. Instantiate a 3D convolutional neural network.
5. Choose a loss function and metrics to use.
6. Train on part of the data.
7. Evaluate on the rest of the data.

## Google Colaboratory

If you are using Colab, please switch your runtime to GPU. To do this, select `Runtime > Change runtime type` in the top menu. Then select GPU under `Hardware accelerator`. A GPU greatly speeds up training.

In [ ]:
import nobrainer

# Get sample features and labels

We use 9 pairs of volumes for training and 1 pair of volumes for evaulation. Many more volumes would be required to train a model for any useful purpose.

In [ ]:
csv_of_filepaths = nobrainer.utils.get_data()
filepaths = nobrainer.io.read_csv(csv_of_filepaths)

train_paths = filepaths[:9]
evaluate_paths = filepaths[9:]

# Convert medical images to TFRecords

Remember how many full volumes are in the TFRecords files. This will be necessary to know how many steps are in on training epoch. The default training method needs to know this number, because Datasets don't always know how many items they contain.

In [ ]:
# Verify that all volumes have the same shape and that labels are integer-ish.

invalid = nobrainer.io.verify_features_labels(train_paths)
assert not invalid

invalid = nobrainer.io.verify_features_labels(evaluate_paths)
assert not invalid

In [ ]:
!mkdir -p data

In [ ]:
# Convert training and evaluation data to TFRecords.

nobrainer.tfrecord.write(
    features_labels=train_paths,
    filename_template='data/data-train_shard-{shard:03d}.tfrec',
    examples_per_shard=3)

nobrainer.tfrecord.write(
    features_labels=evaluate_paths,
    filename_template='data/data-evaluate_shard-{shard:03d}.tfrec',
    examples_per_shard=1)

In [ ]:
!ls data

# Create Datasets

In [ ]:
n_classes = 1
batch_size = 2
volume_shape = (256, 256, 256)
block_shape = (32, 32, 32)
n_epochs = None
shuffle_buffer_size = 10
num_parallel_calls = 2

Take a look at different augmentation options in Nobrainer spatial and intensity transforms. To set training with multiple augmentations, the 'augment' will be set as a list where their order will determine the sequence of execution. For example augment option below will first add Gaussian noise and will then perform the random flip. 
Parameters of any given augmentation techniques can be set as shown below ( eg. 'noise_mean':0.1') otherwise default parameter settings will be applied. 

For training without augmentation, set 'augment = None'. 

In [ ]:
from nobrainer.intensity_transforms import *
from nobrainer.spatial_transforms import *

augment = [(addGaussianNoise, {'noise_mean':0.1,'noise_std':0.5}), (randomflip_leftright)]

In [ ]:
dataset_train = nobrainer.dataset.get_dataset(
    file_pattern='data/data-train_shard-*.tfrec',
    n_classes=n_classes,
    batch_size=batch_size,
    volume_shape=volume_shape,
    block_shape=block_shape,
    n_epochs=n_epochs,
    augment=augment,
    shuffle_buffer_size=shuffle_buffer_size,
    num_parallel_calls=num_parallel_calls,
)

dataset_evaluate = nobrainer.dataset.get_dataset(
    file_pattern='data/data-evaluate_shard-*.tfrec',
    n_classes=n_classes,
    batch_size=batch_size,
    volume_shape=volume_shape,
    block_shape=block_shape,
    n_epochs=1,
    augment=False,
    shuffle_buffer_size=None,
    num_parallel_calls=1,
)

In [ ]:
dataset_train

In [ ]:
dataset_evaluate

# Instantiate a neural network

In [ ]:
model = nobrainer.models.unet(
    n_classes=n_classes, 
    input_shape=(*block_shape, 1),
    batchnorm=True,
)

In [ ]:
model.summary()

# Choose a loss function and metrics

We have many choices of loss functions for binary segmentation. One can choose from binary crossentropy, Dice, Jaccard, Tversky, and many other loss functions.

In [ ]:
import tensorflow as tf

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-04)

model.compile(
    optimizer=optimizer,
    loss=nobrainer.losses.dice,
    metrics=[nobrainer.metrics.dice, nobrainer.metrics.jaccard],
)

# Train and evaluate model

$$
steps = \frac{nBlocks}{volume} * \frac{nVolumes}{batchSize}
$$

In [ ]:
steps_per_epoch = nobrainer.dataset.get_steps_per_epoch(
    n_volumes=len(train_paths),
    volume_shape=volume_shape,
    block_shape=block_shape,
    batch_size=batch_size)

steps_per_epoch

In [ ]:
validation_steps = nobrainer.dataset.get_steps_per_epoch(
    n_volumes=len(evaluate_paths),
    volume_shape=volume_shape,
    block_shape=block_shape,
    batch_size=batch_size)

validation_steps

In [ ]:
model.fit(
    dataset_train,
    epochs=5,
    steps_per_epoch=steps_per_epoch, 
    validation_data=dataset_evaluate, 
    validation_steps=validation_steps)